In [ ]:
import os
os.environ["PYSYN_CDBS"] = '/data/eeh55/Python/synphot/grp/redcat/trds/'

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
import astropy.table as at
from astropy.cosmology import FlatLambdaCDM
from scipy.stats import pearsonr, norm
from snpy import import_lc
import extinction

os.chdir('/data/eeh55/Github/GausSN/py/')
import gausSN
os.chdir('/data/eeh55/Github/bayesn-pre-release/')
from BayeSNmodel import io

plt.style.use('/data/eeh55/Python/stylesheets/SNooPy.mplstyle')

In [ ]:
abnormal_fdr1 = {'ASASSN-16br': '91T', 'PS16ez': '91T', 'ASASSN-16br': '91T', '2016ajf': '91T', 'PS16amf': '06gz',
                 'LSQ16sf': '91T', 'LSQ16te': '91T', '2016aqt': '91T', 'ASASSN-16ci': '91bg', 'ASASSN-16ct': '91T',
                 'ASASSN-16cy': '91T', 'iPTF16abc': '91T', 'ASASSN-16ex': '06gz', 'ASASSN-16fo': '91T',
                 'ASASSN-16hc': '91T', 'ATLAS16bwu': '91T', 'Gaia16bby': '91T?', 'PS16eqv': '91T', 'CSS161013': '91T',
                 'PS16evk': '91T', 'ASASSN-16oz': '91T', 'ATLAS17air': '91T?', 'Gaia17aea': '00cx?; 91bg?',
                 'PTSS-17dgm': '06gz', 'ASASSN-17bd': '91T', 'ATLAS17bas': '91T', 'ASASSN-15bc': '91T',
                 'ASASSN-15fa': '91T', 'ASASSN-15ga': '91bg', 'ASASSN-15hy': '06gz', 'ASASSN-15lu': '91T',
                 'ASASSN-15mi': '91T', 'ASASSN-15nr': '91T', 'ASASSN-15tg': '91T', 'CSS151120': '91T',
                 'MASTEROTJ22': '91bg', 'PS15zn': 'Ic', 'PS15bif': '91T', 'PS15brr': '91T', 'PS15bwh': '91T',
                 'PS15cfn': '91T', 'PS15cwx': '91T', 'PSNJ16283836': '91bg', 'PSNJ23102264': '06gz'}

abnormal_yse = ascii.read('/data/eeh55/Github/yse/data/full/sntypes.csv')
abnormal_yse = abnormal_yse[abnormal_yse['spec_class'] != 'SN Ia']

In [ ]:
ysetab = ascii.read('/data/eeh55/Github/yse/data/full/summary_stats_dropz.txt')
fdr1tab = ascii.read('/data/eeh55/Github/bayesn-pre-release/bayesn-data/lcs/Foundation_DR1/Foundation_DR1/summary_stats_dropz.txt')

ysetab['survey'] = np.repeat(['yse'], len(ysetab))
fdr1tab['survey'] = np.repeat(['fdr1'], len(fdr1tab))

tab = at.vstack([ysetab, fdr1tab])
tab.add_index(['name'])
tab = tab[np.isin(tab['name'], list(abnormal_fdr1.keys()), invert=True)]
tab = tab[np.isin(tab['name'], list(abnormal_yse['\ufeffname']), invert=True)]

obs_before_t0 = [snid for snid in tab['name'] if tab[tab['name'] == snid]['N_zband_before_t0'] > 0]
obs_before_t0_plus4 = [snid for snid in obs_before_t0 if tab[tab['name'] == snid]['N_zband'] > 4]
obs_before_t0_plus4_redshiftcut = [snid for snid in obs_before_t0_plus4 if tab[tab['name'] == snid]['redshift'] < 0.1
                                   and tab[tab['name'] == snid]['redshift'] > 0.01]

In [ ]:
from scipy.stats import norm
def likelihood(parameters, x, y): 
    A = parameters[0]
    B = parameters[1]
    t0 = parameters[2]
    Tfall = parameters[3]
    t1 = parameters[4]
    Trise = parameters[5]
    
    y_exp, cov = gausSN.GP(x, x, y, gausSN.ExpSquaredKernel, kernel_params, gausSN.Karpenka2012, parameters)

    L = np.sum(np.log(norm.pdf(y - y_exp, loc=0)))
    return -L

mean_params = [1, 2, np.min(zband["mjd"])-5, 10, np.min(zband["mjd"])+10, 200]
init_likelihood = likelihood(mean_params, zband["mjd"], zband["mag"])

for i in range(1000):
    A = 10**(np.random.uniform(10**(-5), 10, 1))
    B = 10**(np.random.uniform(10**(-5), 10, 1))
    t0 = np.random.uniform(np.min(zband["mjd"]), np.min(zband["mjd"])+100, 1)
    t1 = np.random.uniform(np.min(zband["mjd"]), np.min(zband["mjd"])+100, 1)
    Trise = np.random.uniform(np.min(zband["mjd"]), np.min(zband["mjd"])+100, 1)
    Tfall = np.random.uniform(np.min(zband["mjd"]), np.min(zband["mjd"])+100, 1)
    
    new_mean_params = [A, B, t0, Tfall, t1, Trise]
    new_likelihood = likelihood(new_mean_params, zband["mjd"], zband["mag"])
    if new_likelihood > init_likelihood and new_likelihood < 10000:
        init_likelihood = new_likelihood
        mean_params = new_mean_params
        
print(mean_params)
print(init_likelihood)

In [ ]:
ysedir = '/data/eeh55/Github/yse/data/full/'
ysefiles = [ysedir+fn for fn in os.listdir(ysedir) if fn.endswith('.snana.dat')]
fdr1dir = '/data/eeh55/Github/bayesn-pre-release/bayesn-data/lcs/Foundation_DR1/Foundation_DR1/'
fdr1files = [fdr1dir+fn for fn in os.listdir(fdr1dir) if fn.endswith('.txt') and
             np.isin(fn, ['summary_stats_dropz.txt', 'salt2_latex_table.txt', 'salt2_latex_table_small.txt', 'salt2_latex_table_normal.txt'], invert=True)]
snana_files = ysefiles + fdr1files

tab.add_columns(cols = [np.zeros(len(tab)), np.zeros(len(tab)), np.zeros(len(tab))],
                names=['zMmax', 'ze_Mmax', 'redchi2'])

for fn in snana_files[0:48]:
    snid, lc = io.read_snana_lcfile(fn)
    
    if np.isin(snid, obs_before_t0_plus4_redshiftcut, invert=True):
        continue
    
    zband = lc[lc['flt'] == 'z']
    zband = zband[zband['fluxcal'] >= 0]
    zband["mag"] = zband["zptmag"] - 2.5*np.log10(zband["fluxcal"])
    zband["magerr"] = np.fabs((2.5/np.log(10))*zband["fluxcalerr"]/zband["fluxcal"])
    
    
        
    times = np.arange(np.min(zband['mjd']), np.max(zband['mjd'])+1, 0.5)
    kernel_params = [1, 40]
    
    expectation, variance = gausSN.GP(times, zband["mjd"], zband["mag"],
                                      gausSN.ExpSquaredKernel, kernel_params, gausSN.Karpenka2012, mean_params)
    std = np.sqrt(np.diag(variance))
    
    
    zMmax, zcov_Mmax = gausSN.GP(tab[tab['name'] == snid]['t0'], zband["mjd"], zband["mag"],
                                 gausSN.ExpSquaredKernel, kernel_params, gausSN.Karpenka2012, mean_params)
    tab.loc[snid]['zMmax'] = zMmax
    tab.loc[snid]['ze_Mmax'] = np.sqrt(np.diag(zcov_Mmax))
    
    expected, cov_expected = gausSN.GP(zband["mjd"], zband["mjd"], zband["mag"],
                                       gausSN.ExpSquaredKernel, kernel_params, gausSN.Karpenka2012, mean_params)
    dof = len(zband)-1
    redchi2 = np.sum((zband['mag'] - expected)**2/(expected)) / dof
    tab.loc[snid]['redchi2'] = redchi2
    
    
    
    plt.figure(figsize=(6,4))
    plt.errorbar(zband['mjd'], zband['mag'], yerr=zband['magerr'], ls='None', marker='.')
    plt.plot(times, expectation, color='dimgray')
    plt.fill_between(times, expectation - std, expectation + std, color='gray', alpha=0.2)
    plt.axvline(tab[tab['name'] == snid]['t0'], ls='--', color='k', label='t0', alpha=0.75)
    plt.gca().invert_yaxis()
    plt.xlabel('mjd', fontsize=14)
    plt.ylabel('z band flux', fontsize=14)
    plt.title(snid, fontsize=18)
    plt.legend()
    plt.show()
    plt.clf()